# Limits of Simplified Ikeda vs. DB

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#from jupyterthemes import jtplot
#jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#from pylab import rcParams
#rcParams['figure.figsize'] = 15, 5

from rolldecay import database
from rolldecay.paper_writing import save_fig
import rolldecayestimators.simplified_ikeda as si

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.1, 
                             exclude_table_name='rolldecay_exclude')
db = database.get_db()

In [ ]:
df_rolldecay.describe()

In [ ]:
si.limits_kawahara

In [ ]:
df_rolldecay.columns

In [ ]:
df_limits = pd.DataFrame()
scale_factor=df_rolldecay['scale_factor']
d = (df_rolldecay['TF']+df_rolldecay['TA'])/2/scale_factor
B = df_rolldecay['beam']/scale_factor
LPP = df_rolldecay['lpp']/scale_factor
kg = df_rolldecay['kg']/scale_factor
OG = -kg + d
omega0 = df_rolldecay['omega0']
bBk=df_rolldecay['BKB']/scale_factor
lBk=df_rolldecay['BKL']/scale_factor

Volume=df_rolldecay['Volume']/(scale_factor**3)
df_limits['CB'] = Volume/(B*d*LPP)
df_limits['B/d'] = B/d
df_limits['CMID'] = df_rolldecay['A0']
df_limits['OG/d']=OG/d
df_limits['OMEGA_hat']=omega0*np.sqrt(B/2/9.81)
df_limits['bBk/B']=bBk/B
df_limits['lBk/LPP']=lBk/LPP

In [ ]:
row = df_limits.iloc[0]
row.index

In [ ]:
def check_limit(row):
    
    limits = si.limits_kawahara.copy()
    limits_valid = pd.Series()
    limits_valid.name = row.name
    
    if pd.isnull(row['lBk/LPP']):
        # No bilge keel:
        limits.pop('lBk/LPP')
        limits.pop('bBk/B')
        limits_valid['lBk/LPP']=True
        limits_valid['bBk/B']=True
       
    for key,values in limits.items():
        limits_valid[key]=(row[key]>=values[0]) and (row[key]<=values[1])

    return limits_valid
    

In [ ]:
df_limits_valid=df_limits.apply(func=check_limit, axis=1)

In [ ]:
def plot_limits(limits):

    fig,axes=plt.subplots(nrows=len(limits))
    
    for (key,values),ax in zip(sorted(limits.items()),axes):
        
        mask = df_limits_valid[key]
        number_of_lost = len(df_limits) - np.sum(mask)
        df_ok = df_limits.loc[mask].copy()
        df_not_ok = df_limits.loc[~mask].copy()
            
        # Plotting:
        N= 20
        #bins = np.linspace(df_limits[key].min(), df_limits[key].max(), N)
        bins = np.linspace(df_limits[key].quantile(0.02), df_limits[key].quantile(0.99), N)
        
        
        df_not_ok.hist(column=key, bins=bins, ax=ax, label='exceedance', color='r')
        df_ok.hist(column=key, bins=bins, ax=ax, label='ok', alpha = 1, color='g')
        
        ylim= ax.get_ylim()
        
        ax.plot([values[0],values[0]],[np.min(ylim),np.max(ylim)],'b--', label='min')
        ax.plot([values[1],values[1]],[np.min(ylim),np.max(ylim)],'m--', label='max')
        
        ax.set_title('Exceeding:%i' % (number_of_lost))
        ax.set_xlabel(key)
        ax.set_ylabel('Number of tests')
    
    axes[0].legend();
    return fig

In [ ]:
fig = plot_limits(limits=si.limits_kawahara)

In [ ]:
limits_kawahara_critical = dict()
limits_kawahara_critical['B/d'] = si.limits_kawahara['B/d']
limits_kawahara_critical['CMID'] = si.limits_kawahara['CMID']

fig = plot_limits(limits=limits_kawahara_critical)

In [ ]:
save_fig(fig=fig, name='si_limits')